In [1]:
%load_ext sql

import pandas as pd
from sqlalchemy import create_engine
import json
pd.options.mode.chained_assignment = None

In [2]:
def read_excel_to_dataframe(file_path, worksheet_name):
    """
    Reads the specified worksheet from an Excel file and returns a Pandas data frame.

    Arguments:
        file_path (str): The path to the Excel file.
        worksheet_name (str): The name of the worksheet to read.

    Returns:
        pandas.DataFrame: The data frame containing the data from the specified worksheet.
    """
    # Read the Excel file
    xls = pd.ExcelFile(file_path)

    # Read the specified worksheet into a data frame
    df = pd.read_excel(xls, worksheet_name)

    return df


In [3]:
# Convert string columns to lowercase
# to make future where clauses easier
def convert_all_strings_to_lc(df):
    string_cols = df.select_dtypes(include=['object']).columns
    df[string_cols] = df[string_cols].apply(lambda x: x.str.lower() if x.name in string_cols else x)
    return df


In [4]:
# convert string field of yyyy-mm-dd to datetime field.
def convert_date_column_to_datetime(table_name, column_name, db_name="timelycare"):

    from sqlalchemy import create_engine, MetaData, Table, Column, Date, String

    engine = create_engine(f'sqlite:///{db_name}.db', echo=False)
    conn = engine.connect()

    # Rename the original table
    conn.execute(f"ALTER TABLE {table_name} RENAME TO {table_name}_old")

    # Create a new table with the same schema, but with the date column as a date type
    metadata = MetaData()
    old_table = Table(f"{table_name}_old", metadata, autoload=True, autoload_with=engine)
    new_table = Table(table_name, metadata,
        *[Column(c.name.lower(), Date()) if c.name == col_name else Column(c.name.lower(), String()) for c in old_table.columns],
    )
    new_table.create(engine)

    # Copy the data from the old table to the new table, transforming the date column
    select_stmt = old_table.select()
    insert_stmt = new_table.insert().from_select(
        [c.name.lower() for c in new_table.columns], select_stmt
    )
    insert_stmt = insert_stmt.on_conflict_do_nothing()
    conn.execute(insert_stmt)

    # Drop the old table
    conn.execute(f"DROP TABLE {table_name}_old")

    # Close the connection
    conn.close()
    

In [5]:
def parse_upload_json_format(i, df, table_name, engine, json_columns):
    if json_columns is not None and json_columns[i] is not None:
        for json_col in json_columns:
            if json_col in df.columns:
                
                # Parse the JSON data and store it in a new data frame
                json_data = df[json_col].apply(json.loads)
                json_df = pd.json_normalize(json_data.explode())

                # Write the JSON data frame to a new table
                json_table_name = json_col
                json_df.to_sql(json_table_name, con=engine, if_exists='replace', index=False)

                # convert_date_column_to_datetime(table_name, json_col)


In [6]:
def write_dataframes_to_database(data_frames, table_names, json_columns=None):
    """
    Writes a list of Pandas data frames to local database tables using SQLAlchemy. 
    If a column name is provided as json_column, the contents of that column will be parsed 
    as a JSON file and uploaded to a new table with the same name as the column.

    Arguments:
        data_frames (list of pd.DataFrames): The data frames to write to the database.
        table_names (list of str): The names of the tables to create or overwrite in the database.
        json_columns (list of str): The names of the columns to parse as JSON files and upload to new tables.
    """

    # Create a SQLAlchemy engine for the local database
    engine = create_engine('sqlite:///timelycare.db')

    # Iterate over each data frame and table name
    for i, (df, table_name) in enumerate(zip(data_frames, table_names)):

        df = convert_all_strings_to_lc(df)
        
        # if json column, pop it out and upload separate.
        parse_upload_json_format(i, df, table_name, engine, json_columns)                    
                    
        # Write the remaining data frame to the specified database table
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)


file_path       = "../data/prompt.xlsx"
worksheet_names = ["visit_table", "member_table", "provider_table"]
table_names     = ["visit", "member", "provider"]
json_columns    = [None, None, "License"]
tables_to_create = []

# Read the specified worksheets into data frames
data_frames = [read_excel_to_dataframe(file_path, sheet) for sheet in worksheet_names]

# Write the data frames to the database, with the specified JSON columns parsed into new tables
write_dataframes_to_database(data_frames, table_names, json_columns)



using https://jupyter-tutorial.readthedocs.io/en/stable/data-processing/postgresql/ipython-sql.html

# SQL Query
*Prompt*: We're noticing a data quality issue and want you to investigate. 

1. What's the total number of visits that cannot occur because the provider was double-booked at the same time?  

2. What is the total count that can not continue because the provider didn't have a valid license in the member's state?

*Approach*: 

1. I will need to find all the the instances in which a provider had a scheduled start time that occured before their prior session's end time.

2. I will need to use the "lisence" table (spin off of the json table I created) joined on teh member table to find all the instances in which a provider *can* provide services. Then I'll filter the main table for each of these combinations. I'll then subtract from the total row count to find the number that *can't*. 

In [7]:

# Connect to the local database
%sql sqlite:///timelycare.db


In [19]:
%%sql

with lag_table as (
  SELECT 
    id as session_id,
    provider_id, 
    LAG(provider_id) OVER (ORDER BY scheduled_date_time) AS prev_provider_id,
    LAG(ended_date_time) OVER (ORDER BY scheduled_date_time) as prev_end_time,
    scheduled_date_time, 
    started_date_time,
    member_id as current_member_id,
    LAG(member_id) OVER (ORDER BY scheduled_date_time) AS prev_member_id
  FROM visit
)

select session_id, provider_id, started_date_time
  FROM lag_table
 WHERE provider_id = prev_provider_id
       and current_member_id != prev_member_id -- assume that if ==, then duplicate record, not truly a double-book.
order by session_id, provider_id, scheduled_date_time

 * sqlite:///timelycare.db
Done.


session_id,provider_id,started_date_time
7,3,2023-01-15 05:48:53.000000
8,3,2023-01-15 05:15:49.000000
13,3,2023-01-14 19:58:02.000000
19,1,2023-01-15 20:00:11.000000
22,3,2023-01-15 19:57:22.000000
25,5,2023-01-15 19:50:00.000000


**Answer to Q1:** There are 6 instances. 

One of them, however, does not have a previous end time. 
This is the duplicate session for Provider 5. 
Since we are investigating potential erroneous data, I choose to include it, because it seems likely that this data is erroneous.

Python version of Q1:

In [9]:
visit_query = %sql SELECT * FROM visit
visit_df = visit_query.DataFrame()

member_query =  %sql SELECT * FROM member
member_df =  member_query.DataFrame()

provider_query = %sql SELECT * FROM provider
provider_df =  provider_query.DataFrame()

license_df_df_query = %sql SELECT * FROM license
license_df = license_query.DataFrame()

 * sqlite:///timelycare.db
Done.
 * sqlite:///timelycare.db
Done.
 * sqlite:///timelycare.db
Done.
 * sqlite:///timelycare.db
Done.


In [10]:

lag_table = visit_df[["id", "provider_id", "scheduled_date_time", "member_id", "ended_date_time"]]
lag_table.columns = ["session_id", "provider_id", "scheduled_date_time", "current_member_id", "prev_end_time"]

lag_table.loc[:, "prev_provider_id"] = lag_table["provider_id"].shift(1)
lag_table.loc[:, "prev_end_time"] = lag_table["prev_end_time"].shift(1)
lag_table.loc[:, "prev_member_id"] = lag_table["current_member_id"].shift(1)

result = lag_table[(lag_table["provider_id"] == lag_table["prev_provider_id"]) & (lag_table["current_member_id"] != lag_table["prev_member_id"])]
result = result[["session_id", "provider_id", "scheduled_date_time", "prev_end_time"]].sort_values(by=["session_id", "provider_id", "scheduled_date_time"])

result

,session_id,provider_id,scheduled_date_time,prev_end_time
7,8,3,2023-01-15 05:11:08.000000,2023-01-15 06:04:47.000000
8,9,3,2023-01-15 04:52:18.000000,2023-01-15 05:40:46.000000
13,14,3,2023-01-14 19:45:00.000000,2023-01-14 20:16:43.000000
18,19,1,2023-01-15 20:00:00.000000,2023-01-15 20:43:43.000000
22,23,3,2023-01-15 19:47:07.000000,2023-01-15 20:38:02.000000


## Question 2

Sql version:

In [11]:
%%sql
SELECT *
FROM visit
WHERE provider_id NOT IN (
    SELECT _id as provider_id
    FROM license
    WHERE state = (
        SELECT visit_state
        FROM member
        WHERE member.id = visit.member_id
    )
);


 * sqlite:///timelycare.db
Done.


id,member_id,provider_id,service_line,scheduled_date_time,started_date_time,ended_date_time
2,1,4,hc,2023-01-17 05:00:00.000000,2023-01-17 05:01:21.000000,2023-01-17 05:41:50.000000
17,5,4,hc,2023-01-14 19:33:59.000000,2023-01-14 19:40:29.000000,2023-01-14 19:43:28.000000
20,3,2,pysch,2023-01-15 20:00:00.000000,2023-01-15 20:00:03.000000,2023-01-15 21:01:03.000000
21,5,4,hc,2023-01-15 20:00:00.000000,2023-01-15 19:58:48.000000,2023-01-15 21:01:03.000000
24,3,4,hc,2023-01-17 04:06:08.000000,2023-01-17 04:14:55.000000,2023-01-17 04:27:11.000000
25,8,5,ther,2023-01-12 19:47:07.000000,2023-01-15 19:50:00.000000,None


The total count is 6. I chose to show all data instead of showing the total count. To show the total count (desireable, for instance, if the row count was very high), replace the first line with `select count(*) as total_count`. 

Python version:

In [15]:
license_df['provider_id'] = license_df['_id'].astype('object')


# Join visit and member tables on member_id column
visit_member = pd.merge(visit_df, member_df, left_on='member_id', right_on='id')
visit_member['provider_id'] = visit_member['provider_id'].astype('object')


# Join license table with the above table on _id and visit_state columns
visit_member_license = pd.merge(visit_member, license_df, left_on=['provider_id', 'visit_state'], right_on=['provider_id', 'state'], how='left')

# Filter the rows where _id is null (not found in license table)
result = visit_member_license[visit_member_license['provider_id'].isnull()]

# Get the count of rows
total_count = len(result)
total_count


,id_x,member_id,provider_id,service_line,scheduled_date_time,started_date_time,ended_date_time,id_y,gender,age,visit_state,_id,type,state,active,exp_date


I couldn't get the python version to work: either I'd have the full 25 rows or 0 rows. 
